In [1]:
!pip -q install getpass4 pypdf openai faiss-cpu llama-index llama-index-readers-file llama-index-llms-ollama llama-index-embeddings-huggingface llama-index-vector-stores-faiss langchain bert-score ragas


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━

Checking for upgrades of openai

## **Environment SetUp**
Solution is developed in google collab

In [2]:
import pandas as pd

In [3]:
import nest_asyncio

nest_asyncio.apply()

In [4]:
import os
from google.colab import userdata

In [5]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [6]:
from llama_index.core import SimpleDirectoryReader, ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.node_parser import (SentenceWindowNodeParser,)
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core import Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore


**Creating the LLM**

In [7]:
llm = OpenAI(temperature=0,
             model="gpt-4o",
             max_tokens=500)

**Creating Embeddings**

In [8]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding(embed_batch_size=10)

In [9]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

# **Loading Data and Creating Indexes**

In [11]:
racism_report = SimpleDirectoryReader(
    input_files=["/content/HR.pdf"]
).load_data()

**Building Indices**

In [12]:
dim = 1536
faiss_index = faiss.IndexFlatL2(dim)

In [13]:
vec_store = FaissVectorStore(faiss_index=faiss_index)
context = StorageContext.from_defaults(vector_store=vec_store)
report_index = VectorStoreIndex.from_documents(racism_report, storage_context=context)

**Building the Query engine**

In [14]:
report_engine = report_index.as_query_engine(similarity_top_k=5)

In the below section the Questions and their Baseline Responses are read from the excel sheet "Questions_and_Baseline_Responses.xlsx". A pd dataframe is created from the file read

In [15]:
file_path = '/content/proposed_responses.xlsx'

In [16]:
question_list=pd.read_excel(file_path)

In [17]:
type(question_list)

pandas.core.frame.DataFrame

In [18]:
question_list.head(5)

,Question,Responses
0,What is the aim of the report study?,The aim of the study is to better understand h...
1,How is racism at work experienced by marginali...,Racism at work can mean being passed over for ...
2,What is the most experienced form of racism at...,Close to half described most experienced form ...
3,What are top 3 solutions proposed by the repor...,"By ensuring, managers are trained to act accor..."


### Retrieving responses from the document using RAG

In the below section appropriate responses to few shot prompt questions are retrieved using RAG model created. The responses are added to a list data[] containing the question, the corresponding baseline response and the RAG generated response. This list[] object is then converted to a pd dataframe, new_data. The objective of this dataframe is to get a comparative view of the baseline response and the RAG generated response which can then be used to calculate the BERT score in the subsequent section

In [19]:
data=[]

In [20]:
for index, row in question_list.iterrows():
    question = row['Question']
    # Assuming you want to use the answer as well, though it's not clear how in this context
    response = row['Responses']
    # Generate a response using the report_engine
    generated_response = report_engine.query(question)
    data.append({
        'Question': question,
        'Response': response,
        'RAG_Response': generated_response
    })
    print(f"Question: {question}")
    print(f"Response:{response}")
    print(f"Generated Response: {generated_response}\n")

Question: What is the aim of the report study?
Response:The aim of the study is to better understand how racism prevails in workplaces and highlight the organizational and systemic factors that contribute to the issue.
Generated Response: The aim of the study is to better understand the prevalence of racism in workplaces and the various ways it manifests. It seeks to highlight the organizational and systemic factors that create environments where racism is commonplace, tolerated, and persistent. The study does not aim to compare experiences across different countries but rather to show that racism and the systems supporting it exist globally.

Question: How is racism at work experienced by marginalised racial and ethnic groups? 
Response:Racism at work can mean being passed over for a promotion, being paid less, and being excluded from advancement opportunities due to the individual belonging to marginalised racial and ethnic groups. It can also mean being targeted with slurs and stere

In [21]:
new_data=pd.DataFrame(data)

In [22]:
pd.set_option('display.max_colwidth', None)

new_data

,Question,Response,RAG_Response
0,What is the aim of the report study?,The aim of the study is to better understand how racism prevails in workplaces and highlight the organizational and systemic factors that contribute to the issue.,"The aim of the study is to better understand the prevalence of racism in workplaces and the various ways it manifests. It seeks to highlight the organizational and systemic factors that create environments where racism is commonplace, tolerated, and persistent. The study does not aim to compare experiences across different countries but rather to show that racism and the systems supporting it exist globally."
1,How is racism at work experienced by marginalised racial and ethnic groups?,"Racism at work can mean being passed over for a promotion, being paid less, and being excluded from advancement opportunities due to the individual belonging to marginalised racial and ethnic groups. It can also mean being targeted with slurs and stereotypes, as well as derisive comments about physical features, dress, and food.","Racism at work is experienced by marginalized racial and ethnic groups in various insidious ways. It can manifest through workplace harassment, such as derogatory remarks, racial slurs, and racist jokes. Individuals may also face employment and professional inequities, including being passed over for promotions, being paid less, and being excluded from advancement opportunities. Additionally, they may encounter racial stereotypes, degrading comments about their physical appearance, and disrespect towards their cultural practices. These experiences contribute to a hostile work environment and can have significant emotional and psychological impacts on those affected."
2,What is the most experienced form of racism at work as per the report?,"Close to half described most experienced form of workplace harassment as racist, slurs, jokers and other derogatory comments. Also stereotypes, as well as derisive comments about physical features, dress, and food are experienced.","The most experienced form of racism at work, as per the report, is workplace harassment."
3,What are top 3 solutions proposed by the report to combat racism at work?,"By ensuring, managers are trained to act accordingly in cases of racism at work by following clear process of investigation and penaliszing racist incident. Training employees to identify microaggressions and\nteaching them how to step in if they witness one.Creating a code of conduct for customers and clients that explicitly states theorganization’s expectations regarding civility, respect, common courtesy, and hurtful comments about race, ethnicity, gender, and\nother personal characteristics.","The report proposes several solutions to combat racism at work. Three key solutions include:\n\n1. **Challenge White Defaults at Work**: This involves creating a workplace climate that respects all races, ethnicities, and cultures equally, addressing both overt and covert harassment, and ensuring accountability when racism occurs.\n\n2. **Create a Climate of Respect**: Encourage a respectful workplace where employees are treated with dignity and civility. Training all employees on the importance of a respectful climate is essential to fostering an environment free of racism.\n\n3. **Establish Accountability Mechanisms**: Develop clear processes for addressing racism, including anonymous reporting mechanisms, guidelines for investigation, and consequences for racist behavior. Ensuring that any retaliation is taken seriously and investigated is also crucial."


### Calculating BERT Score

In the below section a for loop is created to calculate the BERT score for each of the rows in the dataframe new_data, comparing the baseline responses and the RAG generated responses. The generated score is then appended to the data frame to create a new df new_bert which is then converted to excel "Bert_Score.xls" and downloaded.

In [23]:
from bert_score import score

In [24]:
bertScores = []

In [25]:
for _, row in new_data.iterrows():
    question = row["Question"]
    resp_baseline = row["Response"]
    resp_llm = row["RAG_Response"]

    resp_baseline_text = str(resp_baseline)
    resp_llm_text = str(resp_llm)

    # Calculate BERTScore
    P, R, F1 = score([resp_llm_text],[resp_baseline_text],lang="en")
    bertScores.append(F1.item())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

In [26]:
type(bertScores)

list

In [27]:
bertScores_df=pd.DataFrame(bertScores, columns=["BERTScore"])

In [28]:
new_data_bert=pd.concat([new_data,bertScores_df],axis=1)

In [29]:
new_data_bert

,Question,Response,RAG_Response,BERTScore
0,What is the aim of the report study?,The aim of the study is to better understand how racism prevails in workplaces and highlight the organizational and systemic factors that contribute to the issue.,"The aim of the study is to better understand the prevalence of racism in workplaces and the various ways it manifests. It seeks to highlight the organizational and systemic factors that create environments where racism is commonplace, tolerated, and persistent. The study does not aim to compare experiences across different countries but rather to show that racism and the systems supporting it exist globally.",0.931906
1,How is racism at work experienced by marginalised racial and ethnic groups?,"Racism at work can mean being passed over for a promotion, being paid less, and being excluded from advancement opportunities due to the individual belonging to marginalised racial and ethnic groups. It can also mean being targeted with slurs and stereotypes, as well as derisive comments about physical features, dress, and food.","Racism at work is experienced by marginalized racial and ethnic groups in various insidious ways. It can manifest through workplace harassment, such as derogatory remarks, racial slurs, and racist jokes. Individuals may also face employment and professional inequities, including being passed over for promotions, being paid less, and being excluded from advancement opportunities. Additionally, they may encounter racial stereotypes, degrading comments about their physical appearance, and disrespect towards their cultural practices. These experiences contribute to a hostile work environment and can have significant emotional and psychological impacts on those affected.",0.912137
2,What is the most experienced form of racism at work as per the report?,"Close to half described most experienced form of workplace harassment as racist, slurs, jokers and other derogatory comments. Also stereotypes, as well as derisive comments about physical features, dress, and food are experienced.","The most experienced form of racism at work, as per the report, is workplace harassment.",0.879910
3,What are top 3 solutions proposed by the report to combat racism at work?,"By ensuring, managers are trained to act accordingly in cases of racism at work by following clear process of investigation and penaliszing racist incident. Training employees to identify microaggressions and\nteaching them how to step in if they witness one.Creating a code of conduct for customers and clients that explicitly states theorganization’s expectations regarding civility, respect, common courtesy, and hurtful comments about race, ethnicity, gender, and\nother personal characteristics.","The report proposes several solutions to combat racism at work. Three key solutions include:\n\n1. **Challenge White Defaults at Work**: This involves creating a workplace climate that respects all races, ethnicities, and cultures equally, addressing both overt and covert harassment, and ensuring accountability when racism occurs.\n\n2. **Create a Climate of Respect**: Encourage a respectful workplace where employees are treated with dignity and civility. Training all employees on the importance of a respectful climate is essential to fostering an environment free of racism.\n\n3. **Establish Accountability Mechanisms**: Develop clear processes for addressing racism, including anonymous reporting mechanisms, guidelines for investigation, and consequences for racist behavior. Ensuring that any retaliation is taken seriously and investigated is also crucial.",0.843444


In [30]:
new_data_bert.to_excel('Bert_Score.xlsx',index=False)

In [31]:
from google.colab import files

In [32]:
files.download('Bert_Score.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Evaluating Relevacy and Groundedness

In [33]:
retrived_docs = report_engine.retriever.retrieve("What is the aim of the report study?	")

In [34]:
for docs in retrived_docs:
  print(docs.get_text())

9  Consider restorative forms of justice which aim to 
heal victims of racism and discrimination and help 
people understand the harm they have caused so 
they can better repair the injury to the community.30
Intro Key Findings Report Catalyst  
Solution Authors Methodology Acknowledgments Endnotes
At the same time, remove racialized and 
gendered expectations related to leadership skills.39
 9 Create measurement and accountability mechanisms 
to ensure that no racial or gender group is being 
promoted at disproportionate rates.
Intro Key Findings Report Catalyst  
Solution Authors Methodology Acknowledgments Endnotes
9 Four independent coders reviewed their responses and conducted an inductive thematic analysis to 
determine patterns that arose organically from the data and create a coding scheme. Once a coding 
scheme was established, two coders coded a subset of the responses. Once a sufficient level of 
reliability was established, the remainder of the responses were coded by a sin

In [35]:
new_data.RAG_Response

,RAG_Response
0,"The aim of the study is to better understand the prevalence of racism in workplaces and the various ways it manifests. It seeks to highlight the organizational and systemic factors that create environments where racism is commonplace, tolerated, and persistent. The study does not aim to compare experiences across different countries but rather to show that racism and the systems supporting it exist globally."
1,"Racism at work is experienced by marginalized racial and ethnic groups in various insidious ways. It can manifest through workplace harassment, such as derogatory remarks, racial slurs, and racist jokes. Individuals may also face employment and professional inequities, including being passed over for promotions, being paid less, and being excluded from advancement opportunities. Additionally, they may encounter racial stereotypes, degrading comments about their physical appearance, and disrespect towards their cultural practices. These experiences contribute to a hostile work environment and can have significant emotional and psychological impacts on those affected."
2,"The most experienced form of racism at work, as per the report, is workplace harassment."
3,"The report proposes several solutions to combat racism at work. Three key solutions include:\n\n1. **Challenge White Defaults at Work**: This involves creating a workplace climate that respects all races, ethnicities, and cultures equally, addressing both overt and covert harassment, and ensuring accountability when racism occurs.\n\n2. **Create a Climate of Respect**: Encourage a respectful workplace where employees are treated with dignity and civility. Training all employees on the importance of a respectful climate is essential to fostering an environment free of racism.\n\n3. **Establish Accountability Mechanisms**: Develop clear processes for addressing racism, including anonymous reporting mechanisms, guidelines for investigation, and consequences for racist behavior. Ensuring that any retaliation is taken seriously and investigated is also crucial."


In [36]:
from datasets import Dataset

questions = [question for question in new_data.Question]

ground_truths = [response for response in new_data.Response]

answers = [response.response for response in new_data.RAG_Response]

contexts = []

# Inference
for question in questions:
  contexts.append([docs.get_text() for docs in report_engine.retriever.retrieve(str(question))])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}

In [37]:
# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [38]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

In [39]:
df

user_input  \
0                                          What is the aim of the report study?   
1  How is racism at work experienced by marginalised racial and ethnic groups?    
2        What is the most experienced form of racism at work as per the report?   
3     What are top 3 solutions proposed by the report to combat racism at work?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     